In [92]:
import pandas as pd
import numpy as np
import librosa
from librosa import display
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.utils import to_categorical

import time

In [93]:
# Here we will convert all the mp3 files to wav files
import subprocess
import os 

# Get a list of file paths
base_path = 'train_6_audio/'
bird_dirs = os.listdir(base_path)
bird_dirs = [base_path + bird_dir + '/' for bird_dir in bird_dirs]
bird_dirs = list(filter(lambda x: '.' not in x, bird_dirs))
bird_paths = [bird_dir + bird_file for bird_dir in bird_dirs for bird_file in os.listdir(bird_dir)]
wav_paths = [bird_dir + bird_file for bird_dir in bird_dirs for bird_file in os.listdir(bird_dir)]
wav_paths = list(filter(lambda x: '.wav' in x, wav_paths))

for wav_path in wav_paths:
    subprocess.run(['rm', wav_path])

def convert_to_wav(path):
    wav_path = path.strip('.mp3')
    wav_path = wav_path + '.wav'
    subprocess.run(['ffmpeg', '-i', path, wav_path])
    return wav_path

def delete_file(path):
    subprocess.run(['rm', path])
    return

#convert_to_wav(bird_paths)

In [94]:
meta_csv_path = 'train.csv'
meta_data = pd.read_csv(meta_csv_path)
# We drop all audio files with a rating below 4 points, this still leaves some 15 thousand files.
meta_data = meta_data[meta_data.rating >= 4]
# We select the six birds we are working with
choosen_species = set(['amecro', 'grhowl', 'moudov', 'brdowl', 'belin1', 'semplo'])
meta_data = meta_data[meta_data.ebird_code.isin(choosen_species)]
meta_data.shape

(354, 35)

In [137]:
#test_dataset = meta_data.sample(n=10)
#test_dataset.shape
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

def get_fp_from_df(df):
    base_path = "train_audio/"
    codes = df['ebird_code']
    filenames = df['filename']
    path_suffix = list(zip(codes,filenames))
    path_suffix = list(map(lambda x: x[0]+'/'+x[1], path_suffix))
    paths = [base_path + path for path in path_suffix]
    return paths

def get_cluster_averages(df):
    df_t = df.transpose()
    kmeans = KMeans(n_clusters=2)
    y_pred = kmeans.fit_predict(df_t)
    df_t['cluster'] = y_pred.tolist()
    cluster1 = df_t[df_t['cluster'] == 0]
    cluster2 = df_t[df_t['cluster'] == 1]
    cluster1 = cluster1.drop(['cluster'],axis=1)
    cluster2 = cluster2.drop(['cluster'],axis=1)
    c1 = np.mean(cluster1.to_numpy(),axis=0)
    c2 = np.mean(cluster2.to_numpy(),axis=0)
    c1std = np.std(cluster1.to_numpy(),axis=0)
    c2std = np.std(cluster2.to_numpy(),axis=0)
    return np.concatenate((c1,c2,c1std,c2std))

def get_mfccs(path):
    data, sample_rate = librosa.load(path, res_type='kaiser_fast')
    mfcc = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=15)
    mfcc_df = pd.DataFrame(mfcc)
    mfcc_processed = get_cluster_averages(mfcc_df)
    return (mfcc_processed,sample_rate)
    
def extract_features(df):
    mp3_paths = get_fp_from_df(df)
    mfcc_list = []
    librosa_srs = []
    for path in mp3_paths:
        wav_path = convert_to_wav(path)
        extracted = get_mfccs(wav_path)
        mfcc_list.append(extracted[0])
        librosa_srs.append(extracted[1])
        delete_file(wav_path)
    df['mfccs'] = mfcc_list
    df['librosa_sample_rate'] = librosa_srs
    return df

start_time = time.time()
feature_df = extract_features(meta_data)
stop_time = time.time()
print("Feature extraction took {} seconds".format(stop_time-start_time))


Feature extraction took 394.9949064254761 seconds


In [150]:
# Convert features and output labels to numpy arrays
X = np.array(feature_df.mfccs.tolist())
y = np.array(feature_df.ebird_code.tolist())

# Label Encoder
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state=127)

In [151]:
# The actuall feedforward neural network architecture
num_labels = yy.shape[1]
filter_size = 2
def build_model_graph(input_shape=(40,)):
    model = Sequential()
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model

model = build_model_graph()
model.layers
#model.summary()
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: {}".format(accuracy))

Pre-training accuracy: 22.535210847854614


In [158]:
from keras.callbacks import ModelCheckpoint
#from datetime import datetime

num_epochs = 1000
num_batch_size = 354

start_time = time.time()
model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test,y_test),verbose=1)
stop_time = time.time()
print("Training time: {}".format(stop_time-start_time))

Epoch 1/1000
1/1 [==============================] - 0s 42ms/step - loss: 0.0525 - accuracy: 0.9859 - val_loss: 3.5931 - val_accuracy: 0.7183
Epoch 2/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.0468 - accuracy: 0.9859 - val_loss: 3.5809 - val_accuracy: 0.7183
Epoch 3/1000
1/1 [==============================] - 0s 28ms/step - loss: 0.0344 - accuracy: 0.9894 - val_loss: 3.5666 - val_accuracy: 0.7042
Epoch 4/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.0478 - accuracy: 0.9823 - val_loss: 3.5546 - val_accuracy: 0.7042
Epoch 5/1000
1/1 [==============================] - 0s 28ms/step - loss: 0.0272 - accuracy: 0.9894 - val_loss: 3.5443 - val_accuracy: 0.7042
Epoch 6/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.0446 - accuracy: 0.9894 - val_loss: 3.5380 - val_accuracy: 0.7042
Epoch 7/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.0115 - accuracy: 0.9929 - val_loss: 3.5367 - val_accuracy: 0.7183
Epoch 8/1000


Epoch 59/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.0161 - accuracy: 0.9929 - val_loss: 3.5558 - val_accuracy: 0.7042
Epoch 60/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.0724 - accuracy: 0.9929 - val_loss: 3.5457 - val_accuracy: 0.7042
Epoch 61/1000
1/1 [==============================] - 0s 25ms/step - loss: 0.0130 - accuracy: 0.9965 - val_loss: 3.5394 - val_accuracy: 0.7042
Epoch 62/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 3.5317 - val_accuracy: 0.7042
Epoch 63/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.0245 - accuracy: 0.9929 - val_loss: 3.5209 - val_accuracy: 0.7042
Epoch 64/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 3.5122 - val_accuracy: 0.7042
Epoch 65/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.0299 - accuracy: 0.9929 - val_loss: 3.5058 - val_accuracy: 0.7042
Epoch 

1/1 [==============================] - 0s 26ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 3.5859 - val_accuracy: 0.6761
Epoch 117/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.0142 - accuracy: 0.9965 - val_loss: 3.5941 - val_accuracy: 0.6761
Epoch 118/1000
1/1 [==============================] - 0s 25ms/step - loss: 0.0083 - accuracy: 0.9929 - val_loss: 3.5975 - val_accuracy: 0.6761
Epoch 119/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 3.6064 - val_accuracy: 0.6761
Epoch 120/1000
1/1 [==============================] - 0s 27ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 3.6126 - val_accuracy: 0.6761
Epoch 121/1000
1/1 [==============================] - 0s 29ms/step - loss: 0.0076 - accuracy: 0.9965 - val_loss: 3.6130 - val_accuracy: 0.6761
Epoch 122/1000
1/1 [==============================] - 0s 25ms/step - loss: 0.0112 - accuracy: 0.9965 - val_loss: 3.6096 - val_accuracy: 0.6761
Epoch 123/1000

1/1 [==============================] - 0s 24ms/step - loss: 0.0148 - accuracy: 0.9965 - val_loss: 3.5148 - val_accuracy: 0.6761
Epoch 174/1000
1/1 [==============================] - 0s 25ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 3.5099 - val_accuracy: 0.6761
Epoch 175/1000
1/1 [==============================] - 0s 30ms/step - loss: 0.0222 - accuracy: 0.9929 - val_loss: 3.5055 - val_accuracy: 0.6761
Epoch 176/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 3.5025 - val_accuracy: 0.6761
Epoch 177/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.0237 - accuracy: 0.9965 - val_loss: 3.5040 - val_accuracy: 0.6761
Epoch 178/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.0120 - accuracy: 0.9965 - val_loss: 3.5108 - val_accuracy: 0.6761
Epoch 179/1000
1/1 [==============================] - 0s 27ms/step - loss: 0.0416 - accuracy: 0.9894 - val_loss: 3.5192 - val_accuracy: 0.6761
Epoch 180/1000

1/1 [==============================] - 0s 23ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 3.4173 - val_accuracy: 0.7042
Epoch 231/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.0077 - accuracy: 0.9965 - val_loss: 3.4304 - val_accuracy: 0.7042
Epoch 232/1000
1/1 [==============================] - 0s 21ms/step - loss: 7.5206e-04 - accuracy: 1.0000 - val_loss: 3.4413 - val_accuracy: 0.7042
Epoch 233/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 3.4507 - val_accuracy: 0.7042
Epoch 234/1000
1/1 [==============================] - 0s 25ms/step - loss: 0.0067 - accuracy: 0.9965 - val_loss: 3.4608 - val_accuracy: 0.7042
Epoch 235/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.0176 - accuracy: 0.9965 - val_loss: 3.4744 - val_accuracy: 0.7042
Epoch 236/1000
1/1 [==============================] - 0s 25ms/step - loss: 0.0085 - accuracy: 0.9965 - val_loss: 3.4884 - val_accuracy: 0.7042
Epoch 237/

...

1/1 [==============================] - 0s 23ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 3.6086 - val_accuracy: 0.7183
Epoch 744/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.0277 - accuracy: 0.9965 - val_loss: 3.6061 - val_accuracy: 0.7183
Epoch 745/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.0038 - accuracy: 0.9965 - val_loss: 3.6047 - val_accuracy: 0.7042
Epoch 746/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 3.6030 - val_accuracy: 0.7042
Epoch 747/1000
1/1 [==============================] - 0s 18ms/step - loss: 6.6165e-04 - accuracy: 1.0000 - val_loss: 3.6009 - val_accuracy: 0.7042
Epoch 748/1000
1/1 [==============================] - 0s 19ms/step - loss: 0.0109 - accuracy: 0.9894 - val_loss: 3.5970 - val_accuracy: 0.7042
Epoch 749/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.0048 - accuracy: 0.9965 - val_loss: 3.5951 - val_accuracy: 0.7042
Epoch 750/

1/1 [==============================] - 0s 18ms/step - loss: 0.0077 - accuracy: 0.9965 - val_loss: 3.7178 - val_accuracy: 0.7042
Epoch 801/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.0170 - accuracy: 0.9929 - val_loss: 3.7111 - val_accuracy: 0.7042
Epoch 802/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.0099 - accuracy: 0.9965 - val_loss: 3.6871 - val_accuracy: 0.7042
Epoch 803/1000
1/1 [==============================] - 0s 19ms/step - loss: 4.9714e-04 - accuracy: 1.0000 - val_loss: 3.6664 - val_accuracy: 0.7042
Epoch 804/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 3.6494 - val_accuracy: 0.6901
Epoch 805/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 3.6340 - val_accuracy: 0.6901
Epoch 806/1000
1/1 [==============================] - 0s 18ms/step - loss: 8.6994e-04 - accuracy: 1.0000 - val_loss: 3.6212 - val_accuracy: 0.6901
Epoch 

1/1 [==============================] - 0s 23ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 3.6665 - val_accuracy: 0.7183
Epoch 858/1000
1/1 [==============================] - 0s 25ms/step - loss: 7.8000e-04 - accuracy: 1.0000 - val_loss: 3.6702 - val_accuracy: 0.7183
Epoch 859/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.0186 - accuracy: 0.9965 - val_loss: 3.6738 - val_accuracy: 0.7183
Epoch 860/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.0197 - accuracy: 0.9929 - val_loss: 3.6663 - val_accuracy: 0.7183
Epoch 861/1000
1/1 [==============================] - 0s 27ms/step - loss: 0.0102 - accuracy: 0.9965 - val_loss: 3.6614 - val_accuracy: 0.7042
Epoch 862/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.0064 - accuracy: 0.9965 - val_loss: 3.6474 - val_accuracy: 0.7042
Epoch 863/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 3.6360 - val_accuracy: 0.6761
Epoch 864/

1/1 [==============================] - 0s 21ms/step - loss: 5.2775e-04 - accuracy: 1.0000 - val_loss: 3.6447 - val_accuracy: 0.6901
Epoch 915/1000
1/1 [==============================] - 0s 18ms/step - loss: 3.8086e-04 - accuracy: 1.0000 - val_loss: 3.6459 - val_accuracy: 0.6901
Epoch 916/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 3.6488 - val_accuracy: 0.6901
Epoch 917/1000
1/1 [==============================] - 0s 19ms/step - loss: 0.0069 - accuracy: 0.9965 - val_loss: 3.6528 - val_accuracy: 0.6901
Epoch 918/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 3.6565 - val_accuracy: 0.6901
Epoch 919/1000
1/1 [==============================] - 0s 22ms/step - loss: 4.5873e-04 - accuracy: 1.0000 - val_loss: 3.6599 - val_accuracy: 0.6901
Epoch 920/1000
1/1 [==============================] - 0s 28ms/step - loss: 0.0126 - accuracy: 0.9929 - val_loss: 3.6688 - val_accuracy: 0.6901
Ep

1/1 [==============================] - 0s 19ms/step - loss: 3.3440e-04 - accuracy: 1.0000 - val_loss: 3.5533 - val_accuracy: 0.7042
Epoch 972/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 3.5553 - val_accuracy: 0.7042
Epoch 973/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.0104 - accuracy: 0.9965 - val_loss: 3.5571 - val_accuracy: 0.7042
Epoch 974/1000
1/1 [==============================] - 0s 27ms/step - loss: 9.3960e-04 - accuracy: 1.0000 - val_loss: 3.5590 - val_accuracy: 0.7042
Epoch 975/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 3.5607 - val_accuracy: 0.7042
Epoch 976/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.0214 - accuracy: 0.9965 - val_loss: 3.5774 - val_accuracy: 0.7042
Epoch 977/1000
1/1 [==============================] - 0s 22ms/step - loss: 2.0007e-04 - accuracy: 1.0000 - val_loss: 3.5949 - val_accuracy: 0.7042
Ep

In [157]:
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))

Training Accuracy: 100.00%
Testing Accuracy: 71.83%
